In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

import soaplite
import soaputils as su
import genBasis
import ase
from ase.visualize import view
from ase.build import bulk
import numpy as np
from numpy.linalg import svd
import matplotlib as matplotlib
import matplotlib.pyplot as p
import numpy.linalg as nl
from scipy.optimize import fmin
import scipy.optimize as op
import time
from random import shuffle
%matplotlib inline

In [2]:
rCut = 10.0
NradBas = 5
Lmax = 5
myAlphas, myBetas = genBasis.getBasisFunc(rCut, NradBas)

In [3]:
struct2 = bulk('Cu', 'fcc', a=3.6, cubic=True)
struct2l = soaplite._get_supercell(struct2, rCut=2.0)
struct2lr = su.rand_pos(struct2l)
#view(struct2)
#view(struct2l)
mat2 = soaplite.get_periodic_soap_structure(struct2, myAlphas, myBetas, rCut, NradBas, Lmax)
mat2l = soaplite.get_periodic_soap_structure(struct2l, myAlphas, myBetas, rCut, NradBas, Lmax)
mat2lr = soaplite.get_periodic_soap_structure(struct2lr, myAlphas, myBetas, rCut, NradBas, Lmax)
#p.matshow(mat2)
#p.matshow(mat2l)

In [4]:
# l1_0 Au-Cu
a0 = 4.0
#c0 = 0.9 * a0 #https://link.springer.com/content/pdf/10.1007/BF03215214.pdf
c0 = 1.0 * a0
struct3 = bulk('Au', 'fcc', a=a0, cubic=True)
struct3.set_cell(np.diag((a0,a0,c0)), scale_atoms=True)
struct3.set_chemical_symbols(('Au','Cu','Cu','Au'))
struct3l = soaplite._get_supercell(struct3, rCut=2.0)
#view(struct3)
#view(struct3l)
mat3 = soaplite.get_periodic_soap_structure(struct3, myAlphas, myBetas, rCut, NradBas, Lmax)
mat3l = soaplite.get_periodic_soap_structure(struct3l, myAlphas, myBetas, rCut, NradBas, Lmax)
#p.matshow(mat3)
#p.matshow(mat3l)

In [5]:
struct4 = ase.io.read(currentdir + "/glass_cube_CuZr.cfg")
struct4r = su.lim_overlap(su.rand_pos(struct4), dmin=1.5) # randomizing positions and limiting overlap
struct4rb = su.lim_overlap(su.rand_pos(struct4), dmin=2.0)
mat4 = soaplite.get_periodic_soap_structure(struct4, myAlphas, myBetas, rCut, NradBas, Lmax)
mat4r = soaplite.get_periodic_soap_structure(struct4r, myAlphas, myBetas, rCut, NradBas, Lmax)
mat4rb = soaplite.get_periodic_soap_structure(struct4rb, myAlphas, myBetas, rCut, NradBas, Lmax)
#p.matshow(mat4)
#p.matshow(mat4r)
#p.matshow(mat4rb)

In [6]:
#randomly assigned symbols fcc 
struct5 = bulk('Au', 'fcc', a=4.0, cubic=True)
struct5.set_chemical_symbols(('Au','Cu','Cu','Au'))
struct5l = soaplite._get_supercell(struct3, rCut=2.0)
symbols5 = struct5l.get_chemical_symbols()
np.random.shuffle(symbols5)
struct5l.set_chemical_symbols(symbols5)
#view(struct5l)
mat5l = soaplite.get_periodic_soap_structure(struct5l, myAlphas, myBetas, rCut, NradBas, Lmax)
#p.matshow(mat5)

In [7]:
%matplotlib

s1 = svd(np.transpose(mat4), full_matrices=False, compute_uv=False)
s1r = svd(np.transpose(mat4r), full_matrices=False, compute_uv=False)
s2 = svd(np.transpose(mat2l), full_matrices=False, compute_uv=False) # Cu fcc
s2r = svd(np.transpose(mat2lr), full_matrices=False, compute_uv=False) # Cu randomized
s3 = svd(np.transpose(mat3l), full_matrices=False, compute_uv=False) # l1_0 Au-Cu
s4 = svd(np.transpose(mat5l), full_matrices=False, compute_uv=False) # random assigned Au-Cu fcc

plot1, = p.semilogy(s1, label='Cu-Zr Cube')
plot1r, = p.semilogy(s1r, label='Cu-Zr randomized Cube')
plot2, = p.semilogy(s2, label='Cu fcc')
plot2r, = p.semilogy(s2r, label='Cu randomized')
plot3, = p.semilogy(s3, label='l1_0 Au_Cu')
plot4, = p.semilogy(s4, label='random Au_Cu fcc')
p.legend(handles=[plot1, plot1r, plot2, plot2r, plot3, plot4])
p.show

Using matplotlib backend: TkAgg


<function matplotlib.pyplot.show>

In [8]:
s3d = [abs(i-300) for i in s3]
s4d = [abs(i-300) for i in s4]
p.semilogy(s3d)
p.semilogy(s4d)

In [9]:
print(nl.norm(s3))
print(nl.norm(s4))
s3l = np.log(s3)
s4l = np.log(s4)
print(np.sum(s3l))
print(np.sum(s4l))

5199.700674115753
5224.276801261812
-4178.623350623104
-560.6465798077912


In [15]:
sigma = 2
eps = 1
print(nl.norm(mat2l))
print(su.cost_LJ(struct2l,sigma,eps))
print(nl.norm(mat2lr))
print(su.cost_LJ(struct2lr,sigma,eps))

20960.255432293656
-2119.005536495317
21006.009366678325
46349383114.121155
